 데이터 수집, 분석의 관점에서 스크랩한 정보를 저장해야 한다.
 
 
 저장하는 방법에는 *1)**참조를 저장**하는 방법*과, *2)**파일 자체를 저장**하는 방법*이 있다. 첫번째 방법은 파일이 위치한 URL을 저장하는 것이다. 그러나 파일을 한두 번 이상 실제로 보거나 읽어야 한다면, 두 번째 방법을 사용하는 것이 좋다.
 
---

 
 # 파일 저장

## 1. 미디어 파일 저장

* 기초 : 내려 받을 파일이 하나이고, 이름과 확장자를 어떻게 정할지 알고 있을 때.
* 최종 : 기초 ver. 확장하여 특정 속성이 있는 태그에 연결된 내용을 모두 내려받음.

### 기초 ver.
* `urllib.request.urlretrieve` : 원격 URL의 파일 다운로드.
* 해당 디렉토리에 `logo.jpg`라는 이름으로 저장됨.
* `.find` 메서드를 썼기 때문에, 여러 장의 이미지 중 첫 이미지만 다운로드된다.

In [7]:
# module import
from urllib.request import urlretrieve
from urllib.request import urlopen
from bs4 import BeautifulSoup

# 이미지 파일 다운로드 : src 속성 `img`
html = urlopen("http://www.pythonscraping.com/")
bs = BeautifulSoup(html, 'html.parser') # BeautifulSoup 객체 생성
imageLocation = bs.find('a', {"id" : "logo"}).find('img')['src']     # id가 logo인 a 태그 아래에
                                             # 'img'라는 태그를 찾고, 그 이미지에서 src 속성을 가져온다.
urlretrieve(imageLocation, 'logo.jpg') # 위의 이미지 경로를 저장. 첫 번째 이미지만 찾아 저장한다.

('logo.jpg', <http.client.HTTPMessage at 0x1f85030a2e8>)

### 종합 ver.
* 홈페이지에서 해당 속성이 있는 태그에 연결된 파일을 모두 내려받는다.
    - 람다 함수 사용해 `src` 속성 모두 선택.
    - URL 손질하여 절대 경로로 바꾸고 내려 받을 준비를 한다.
    - 컴퓨터의 `downloaded` 폴더 안에 경로를 유지하면서 내려 받는다.
* `os` 모듈 : 파이썬과 운영체제 사이의 인터페이스 역할(파일 경로 조작, 디렉터리 생성, 실행 중인 프로세스와 환경변수에 관한 정보 등)
    - 각 파일이 저장될 디렉터리가 있는지 확인.
    - 없다면 디렉터리를 생성한다.
    

In [12]:
# module import
import os
from urllib.request import urlretrieve
from urllib.request import urlopen
from bs4 import BeautifulSoup

# 파일 다운로드할 경로
download_dir = "downloaded" # 폴더
baseUrl = "http://pythonscraping.com/" # 다운로드할 홈페이지

# URL 손질 후 절대경로 얻는 함수
# 애초에 홈페이지가 http://pythonscraping.com이다. 그런데 이미지 src에는 http://www.로 시작하니까 바꿔야.
def getAbsoluteUrl(baseUrl, source): # source가 뭐지?
    if source.startswith("http://www.") : # 이미지 저장되어 있는 경로가 절대 http://www로 시작한다면
        url = "http://{}".format(source[11:]) # www 뒤의 경로만 받는다.
    elif source.startswith("http://") : # www.없이 그냥 http로 바로 시작하면 그게 절대경로.
        url = source
    elif source.startswith("www."):
        url = source[4:] # www 뒤의 부분만 url로 받는다.
        url = "http://{}".format(source)
    else:
        url = "{}/{}".format(baseUrl, source)
    if baseUrl not in url: # url 오류
        return None
    return url

# 다운로드할 경로 얻는 함수
def getDownloadPath(baseUrl, absoluteUrl, downloadDirectory): # 다운로드 경로
    path = absoluteUrl.replace('www.', '') # www. 없애기
    path = path.replace(baseUrl, '') # url도 없애기
    path = download_dir + path
    directory = os.path.dirname(path) # 디렉토리 형성 : Q) "./" 이런거 안 해도 되나?????
    
    if not os.path.exists(directory): # 디렉토리 없으면 만들고
        os.makedirs(directory)
        
    return path

html = urlopen("http://www.pythonscraping.com") # html 열 때는 baseUrl 여는 게 아니다!
bs = BeautifulSoup(html, 'html.parser')
downloadList = bs.findAll(src = True) # 'src' 속성이 있는 애들만 다운로드!

for download in downloadList:
    fileUrl = getAbsoluteUrl(baseUrl, download['src'])
    if fileUrl is not None:
        print(fileUrl)

urlretrieve(fileUrl, getDownloadPath(baseUrl, fileUrl, download_dir))

http://pythonscraping.com/misc/jquery.js?v=1.4.4
http://pythonscraping.com/misc/jquery.once.js?v=1.2
http://pythonscraping.com/misc/drupal.js?q4na2g
http://pythonscraping.com/sites/all/themes/skeletontheme/js/jquery.mobilemenu.js?q4na2g
http://pythonscraping.com/sites/all/modules/google_analytics/googleanalytics.js?q4na2g
http://pythonscraping.com/sites/default/files/lrg_0.jpg
http://pythonscraping.com//https://covers.oreillystatic.com/images/0636920078067/lrg.jpg
http://pythonscraping.com/img/lrg%20(1).jpg


('downloadedimg/lrg%20(1).jpg', <http.client.HTTPMessage at 0x1f851084e80>)

#### 공부해볼 것.
* http://pythonscraping.com/sites/default/files/lrg_0.jpg :다운로드 경로 downloadedsites/default/files
* http://pythonscraping.com/img/lrg%20(1).jpg : 다운로드 경로 downloadedimg

In [18]:
# 오류
# 모든 이미지 파일 다 받을 생각으로 넣었는데 오류 난다.
for download in downloadList:
    fileUrl = getAbsoluteUrl(baseUrl, download['src'])
    if fileUrl is not None:
        print(fileUrl)
    try:
        urlretrieve(fileUrl, getDownloadPath(baseUrl, fileUrl, download_dir))
        print("이미지 다운로드 성공")
    except Exception as e:
        print(e)
        


http://pythonscraping.com/misc/jquery.js?v=1.4.4
[Errno 22] Invalid argument: 'downloadedmisc/jquery.js?v=1.4.4'
http://pythonscraping.com/misc/jquery.once.js?v=1.2
[Errno 22] Invalid argument: 'downloadedmisc/jquery.once.js?v=1.2'
http://pythonscraping.com/misc/drupal.js?q4na2g
[Errno 22] Invalid argument: 'downloadedmisc/drupal.js?q4na2g'
http://pythonscraping.com/sites/all/themes/skeletontheme/js/jquery.mobilemenu.js?q4na2g
[Errno 22] Invalid argument: 'downloadedsites/all/themes/skeletontheme/js/jquery.mobilemenu.js?q4na2g'
http://pythonscraping.com/sites/all/modules/google_analytics/googleanalytics.js?q4na2g
[Errno 22] Invalid argument: 'downloadedsites/all/modules/google_analytics/googleanalytics.js?q4na2g'
http://pythonscraping.com/sites/default/files/lrg_0.jpg
이미지 다운로드 성공
http://pythonscraping.com//https://covers.oreillystatic.com/images/0636920078067/lrg.jpg
[WinError 123] 파일 이름, 디렉터리 이름 또는 볼륨 레이블 구문이 잘못되었습니다: 'downloaded/https:'
http://pythonscraping.com/img/lrg%20(1).jpg
이미지

## 2. 데이터를 csv로 저장

* `.csv` 파일 : 쉼표로 구분된 파일. 각 행은 줄바꿈 문자로, 열은 쉼표로 구분한다.
* `csv` 라이브러리

---
파이썬은 파일을 만들 때 에러를 일으키지 않는다. 즉, 이미 존재한다면 경고 없이 새 데이터로 덮어 써서 만든다. 

---
HTML 테이블 가져와서 CSV 파일 만들기를 할 때, 사실, 한 번만 작업을 수행하면 될 때는 굳이 함수화하지 않고, 엑셀이나 스프레드시트에 붙여넣어서 작업할 수도 있다.


* 기초 ver
    - `html` 테이블을 가져와서 csv 파일로 만든다.
    - 다른 태그들은 제거하지 않고 만든다.
* 종합 ver
    - `BeautifulSoup`와 `get_text()` 함수를 사용해 태그를 제거한다.


### 기초 ver

In [20]:
# module import
import csv

# csv 파일 읽기, 쓰기 모드. 없기 때문에 생성한다.
csvFile = open('test.csv', 'w+')
try:
    writer = csv.writer(csvFile)
    writer.writerow(("number", "number +2", "number x2"))
    for i in range(10):
        writer.writerow( (i, i+2, i*2))
finally: # 예외의 발생 여부와 관계없이 항상 실행되는 절
    csvFile.close() 

### 종합 ver

* 위키피디아의 텍스트 에디터 비교 페이지에서 HTML 테이블을 CSV 파일로 가져온다.


In [23]:
# module import
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen("https://en.wikipedia.org/wiki/Comparison_of_text_editors")
bs = BeautifulSoup(html, 'html.parser')

# 현재 페이지의 첫 번째 테이블 가져온다.
table = bs.findAll('table', {'class' : 'wikitable'})[0] # 첫 번째
rows = table.findAll('tr') # 각 행의 태그는 tr이다.

# csv파일로 읽는다.
csvFile = open("editors.csv", "wt+") # 없으면 만든다.
writer = csv.writer(csvFile)
try:
    for row in rows:
        csvRow = []
        for cell in row.findAll(['td', 'th']) : # 각 행에서 td, th 칸에 있는 텍스트들이 표의 글자 내용이다.
            csvRow.append(cell.get_text())
            writer.writerow(csvRow)
finally:
    csvFile.close()
            

UnicodeEncodeError: 'cp949' codec can't encode character '\u2011' in position 78: illegal multibyte sequence